# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.74,75,100,4.74,TK,1701786304
1,1,qaqortoq,60.7167,-46.0333,-5.91,68,96,4.76,GL,1701786306
2,2,ust-nera,64.5667,143.2000,-46.71,98,96,2.71,RU,1701786308
3,3,margaret river,-33.9500,115.0667,14.60,80,15,5.35,AU,1701786165
4,4,grytviken,-54.2811,-36.5092,2.94,66,53,5.16,GS,1701786312


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [61]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points('Lng','Lat',
                                  geo = True,
                                  tiles='OSM',
                                 color ='City',
                                 frame_width = 750,
                                 frame_height = 500,
                                 size = 'Humidity',
                                 title ='City Data Frame Display')

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [45]:
# Narrow down cities that fit criteria and drop any results with null values
# criteria list : cloudiness less than 20%, max temp is more than 25C and humidity is below 50
df = city_data_df.loc[(city_data_df['Cloudiness'] < 20) & (city_data_df['Max Temp']>25) & (city_data_df['Humidity']<50)]

# Drop any rows with null values
df.dropna()
# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,99,broken hill,-31.9500,141.4333,30.15,26,14,5.85,AU,1701786489
158,158,curvelo,-18.7564,-44.4308,34.32,34,0,1.54,BR,1701786563
223,223,tamanrasset,22.7850,5.5228,26.95,8,0,1.03,DZ,1701786720
241,241,atakpame,7.5333,1.1333,34.47,22,3,1.37,TG,1701786750
246,246,tillaberi,14.2117,1.4531,35.81,9,2,3.05,NE,1701786762


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

C:\Users\tadip\AppData\Local\Temp\ipykernel_9148\2317004114.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
99,broken hill,AU,-31.9500,141.4333,26,
158,curvelo,BR,-18.7564,-44.4308,34,
223,tamanrasset,DZ,22.7850,5.5228,8,
241,atakpame,TG,7.5333,1.1333,22,
246,tillaberi,NE,14.2117,1.4531,9,
354,bubaque,GW,11.2833,-15.8333,45,
448,jalqani,SD,12.4486,34.2186,19,
505,chabahar,IR,25.2919,60.6430,36,
527,rawson,AR,-43.3002,-65.1023,18,
563,viedma,AR,-40.8135,-62.9967,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"   
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
broken hill - nearest hotel: Ibis Styles
curvelo - nearest hotel: Hotel Bandeirante
tamanrasset - nearest hotel: فندق الأمان
atakpame - nearest hotel: ROCK HOTEL
tillaberi - nearest hotel: Hotel Djoliba
bubaque - nearest hotel: Chez Julio
jalqani - nearest hotel: No hotel found
chabahar - nearest hotel: هتل سپیده
rawson - nearest hotel: Hotel Deportivo
viedma - nearest hotel: Níjar
gao - nearest hotel: No hotel found
sao miguel do araguaia - nearest hotel: Hotel São Miguel


,City,Country,Lat,Lng,Humidity,Hotel Name
99,broken hill,AU,-31.9500,141.4333,26,Ibis Styles
158,curvelo,BR,-18.7564,-44.4308,34,Hotel Bandeirante
223,tamanrasset,DZ,22.7850,5.5228,8,فندق الأمان
241,atakpame,TG,7.5333,1.1333,22,ROCK HOTEL
246,tillaberi,NE,14.2117,1.4531,9,Hotel Djoliba
354,bubaque,GW,11.2833,-15.8333,45,Chez Julio
448,jalqani,SD,12.4486,34.2186,19,No hotel found
505,chabahar,IR,25.2919,60.6430,36,هتل سپیده
527,rawson,AR,-43.3002,-65.1023,18,Hotel Deportivo
563,viedma,AR,-40.8135,-62.9967,23,Níjar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
map1 = hotel_df.hvplot.points('Lng','Lat',
                              geo = True,
                              tiles='OSM',
                             color ='City',
                             frame_width = 750,
                             frame_height = 500,
                             size = 'Humidity',
                             hover_cols = ['Country','Hotel Name'])

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)